In [ ]:
!pip3 install duckdb
!pip3 install pandas
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install jupysql pandas matplotlib duckdb-engine
!pip3 install altair

In [ ]:
import duckdb
import pandas as pd
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb
%sql INSTALL spatial; LOAD spatial;
%config SqlMagic.displaylimit = None


In [ ]:
%%sql
DROP TABLE IF EXISTS Oxford_data;
CREATE TABLE IF NOT EXISTS Oxford_data AS
SELECT *
FROM read_csv('../INF-28546-Churchill Theatres Jan to Sept 2024.csv', types={'Early or Late Start (minutes)': 'VARCHAR', 'WeekBeginning': 'VARCHAR'}, timestampformat='%d/%m/%Y %H:%M')


### Removes the unnecessary columns from the dataset

In [ ]:
%%sql 
DROP TABLE IF EXISTS Oxford_parsed_data;
CREATE TABLE Oxford_parsed_data AS     
-- DESCRIBE
WITH Oxford_data_columns AS (
    SELECT * EXCLUDE (
        "Data Quality Flag (Calculation)",
        "Fiscal Year",
        "zero",
        "Theatres Project Link",
        "Visit Date Filter",
        "Period (Overview/Run/bar/box)",
        "-- create Empty",
        "InfoTeamLink",
        "Period chooser",
        "Period chooser- From",
        "Period chooser- To",
        "Anaethetic Start Time",
        "Anaesthetic",
        "Cases Cancelled", 
        "Data Quality Flag (Field Checks)",
        "Late Starts (>15m) (%)",
        "StaffRole",
        "Show/Hide Cases",
        "Show/Hide based on date range",
        "Show/Hide based on DQ",
        "Show/Hide based on date range (day)",
        "Show/Hide Efficency/Utilisation",
        "Show/Hide present day",
        "NB filter caveats text",
        "Exclude low case #"
    )
    FROM Oxford_data
),
Oxford_data_remove_visit_columns AS (
    SELECT COLUMNS(c -> c NOT SIMILAR TO '(Visit ).*')
    FROM Oxford_data_columns
),
Oxford_data_remove_session_statistics AS (
    SELECT COLUMNS(c -> c NOT SIMILAR TO '.*\(%\).*')
    FROM Oxford_data_remove_visit_columns
),
Oxford_data_remove_sessions AS (
    SELECT COLUMNS(c -> c NOT SIMILAR TO '.*\{session\}.*')
    FROM Oxford_data_remove_session_statistics
)
SELECT DISTINCT *
FROM Oxford_data_remove_sessions
-- WHERE column_name LIKE '%Show%'


In [ ]:
%%sql
DESCRIBE 
Oxford_parsed_data

### Calculates the Overtime

In [ ]:
%%sql 
DROP TABLE IF EXISTS Oxford_data_turnovertime;
CREATE TABLE IF NOT EXISTS Oxford_data_turnovertime  AS 
WITH Oxford_date_case_information AS (
    SELECT 
        "Theatre",
        "Session ID",
        VisitDate,
        "Theatre Case Number",
        "Planned Session Start",
        "Actual Session Start",
        "Planned Session End",
        "Actual Session End",
        "Planned Case Start",
        "Case Start Date Time",
        "Planned Case End",
        "Case End Date Time",
        "CaseTime (minutes)",
        "Theatre Cases per Session (#)",
        "ConsultantSpecialty (group)",
        "Consultant Specialty",
        "Case Type",
        "Consultant In Charge Of Procedure"
    FROM Oxford_parsed_data
    WHERE
        1 = 1
        AND "CaseTime (minutes)" IS NOT NULL 
        AND "Session Type" = 'Planned' 
        AND "Case Type" = 'Elective'
        AND Cancelled = 'N'
        AND "CaseTime (minutes)" > 0

),
Oxford_prev_end_case AS (
    SELECT lag("Case End Date Time") OVER (PARTITION BY VisitDate, "Session Id" ORDER BY "Case Start Date Time") AS actual_previous_end_case_datetime, *
    FROM Oxford_date_case_information
),
Oxford_prev_end_case_filter AS (
    SELECT *
    FROM Oxford_prev_end_case
    WHERE 1 = 1
    AND actual_previous_end_case_datetime IS  NULL 
    OR ("Case Start Date Time" >= actual_previous_end_case_datetime OR "Case End Date Time" >= actual_previous_end_case_datetime)
),
Oxford_data_turnover_time AS (
    SELECT * EXCLUDE actual_previous_end_case_datetime, 
        lag("Case End Date Time") OVER (PARTITION BY VisitDate, "Session Id" ORDER BY "Case Start Date Time") AS actual_previous_end_case_datetime,
        lag("Planned Case End") OVER (PARTITION BY VisitDate, "Session Id" ORDER BY "Planned Case Start") AS expected_previous_end_case_datetime,
        "Case Start Date Time" - actual_previous_end_case_datetime AS actual_turnovertime,
        "Planned Case Start" - expected_previous_end_case_datetime AS expected_turnovertime,
        row_number() OVER (PARTITION BY VisitDate, "Session Id" ORDER BY "Case Start Date Time" ASC) AS actual_day_sequence_number, 
        row_number() OVER (PARTITION BY VisitDate, "Session Id" ORDER BY "Planned Case Start") AS expected_day_sequence_number, 
        hour(actual_turnovertime) * 60 + minute(actual_turnovertime) AS actual_turnovertime_minutes,
        hour(expected_turnovertime) * 60 + minute(expected_turnovertime) AS expected_turnovertime_minutes,
    FROM Oxford_prev_end_case_filter
),
Oxford_data_filter_consult_speciality AS (
    SELECT *
    FROM Oxford_data_turnover_time
    WHERE 1=1
        AND "ConsultantSpecialty (group)" NOT IN (
            'Endoscopy',
            'ENT',
            'Renal Surgery',
            'Trauma & Orthopaedics',
            'Radiology',
            'Unknown',
            'Anaesthetics',
            'Pain management'
        )
),
Oxford_data_split_surgeon AS (
    SELECT *
        , string_split("Consultant In Charge Of Procedure", ', ') AS Oxford_data_split_surgeon
    FROM Oxford_data_filter_consult_speciality
),
Oxford_data_format AS (
    SELECT 
        try_cast(NULL AS NVARCHAR) AS organization_id,
        try_cast(NULL AS NVARCHAR) AS organization_name,
        Theatre AS theatre,
        "Theatre Case Number" AS case_key,
        "Session ID" AS session_key,
        actual_day_sequence_number,
        expected_day_sequence_number,
        Oxford_data_split_surgeon[1] AS surgeon_surname,
        Oxford_data_split_surgeon[2] AS surgeon_name,
        "ConsultantSpecialty (group)" AS specialism_name,
        try_cast(NULL AS nvarchar) AS procedure_id, 
        try_cast(NULL AS nvarchar) AS procedure_name,
        try_cast(NULL AS nvarchar) AS protocol_id,
        try_cast(NULL AS nvarchar) AS protocol_code,
        try_cast(NULL AS nvarchar) AS protocol_name, 
        CAST(VisitDate AS Date) AS date, 
        "Case Start Date Time" AS actual_start_case_datetime,
        "Planned Case Start" AS expected_start_case_datetime,
        "Case End Date Time" AS actual_end_case_datetime,
        "Planned Case End" AS expected_end_case_datatime,
        expected_previous_end_case_datetime,
        actual_previous_end_case_datetime,
        actual_turnovertime,
        expected_turnovertime,
        actual_turnovertime_minutes,
        expected_turnovertime_minutes
    FROM Oxford_data_split_surgeon
)
SELECT * 
FROM Oxford_data_format
-- WHERE Theatre = 'J-TH Main 19' AND date = '2024-04-23 00:00:00'::DATE
--WHERE Theatre = 'J-TH Main 19' AND VisitDate = '2024-04-23 00:00:00'::DATE
--ORDER BY expected_day_sequence_number	


In [ ]:
%%sql
SELECT *
FROM Oxford_data_turnovertime
WHERE theatre = 'Room 11' AND date = '2024-03-25'::DATE

In [ ]:
%%sql 
COPY Oxford_data_turnovertime TO '../Oxford_data_turnovertime_v1.csv'

In [ ]:
%%sql
DROP TABLE IF EXISTS Oxford_parsed_efficiency_data;
CREATE TABLE Oxford_parsed_efficiency_data AS
WITH Oxford_data_session_join AS (
    SELECT 
        DISTINCT 
        Theatre AS theatre
        ,CAST("Actual Session Start" AS DATE) AS date
        ,"Session ID" AS session_key
        ,"Actual Session Start" AS actual_start_case_datetime
        ,"Actual Session End" AS actual_end_case_datetime
        , "Planned Session Start" AS expected_start_case_datetime
        , "Planned Session End" AS expected_end_case_datetime
    FROM Oxford_parsed_Data 
),
Oxford_data_day_filter AS (
    SELECT *
    FROM Oxford_data_session_join
    WHERE 
        (date::DATETIME + INTERVAL 6 HOUR) <= expected_start_case_datetime AND
        (date::DATETIME + INTERVAL 10 HOUR) >= expected_start_case_datetime AND
        (date::DATETIME + INTERVAL 14 HOUR) <= expected_end_case_datetime AND
        (date::DATETIME + INTERVAL 18 HOUR) >= expected_end_case_datetime 
    
),
Oxford_data_grouped_session AS (
    SELECT "Session ID" AS session_key 
        , SUM(
            "Case End Date Time" > "Planned Session End" AND 
            "Case Start Date Time" < "Planned Session End"
        ) AS number_of_records_day_end
    FROM Oxford_parsed_Data
    WHERE actual_start_case_datetime IS NOT NULL
    GROUP BY schedule_key
),
Oxford_data_filter AS (
    SELECT *
    FROM Oxford_data_day_filter 
    WHERE actual_start_case_datetime IS NOT NULL
),
Oxford_data_difference_day AS (
    SELECT 
        OXDS.*,
        IF(
            actual_start_case_datetime < expected_start_case_datetime, 
            expected_start_case_datetime - actual_start_case_datetime, 
            actual_start_case_datetime - expected_start_case_datetime
        ) AS difference_day_start,
        IF(
            actual_end_case_datetime < expected_end_case_datetime, 
            expected_end_case_datetime - actual_end_case_datetime, 
            actual_end_case_datetime - expected_end_case_datetime) AS difference_day_end,
        IF(
            actual_start_case_datetime < expected_start_case_datetime, 
            'Before', 
            'After'
        ) AS day_start_timeline_type,
        IF(actual_end_case_datetime < expected_end_case_datetime, 'Before', 'After') AS day_end_timeline_type,
        (expected_start_case_datetime < actual_start_case_datetime)::INT AS day_start_overtime_indicator,
        (actual_start_case_datetime < expected_end_case_datetime AND expected_end_case_datetime < actual_end_case_datetime)::INT AS day_end_overtime_indicator
        FROM Oxford_data_filter OXDS
),
Oxford_data_difference_minutes AS (
    SELECT 
        *,
        minute(difference_day_start) + hour(difference_day_start) * 60 AS difference_minutes_day_start,
        minute(difference_day_end) + hour(difference_day_end) * 60 AS difference_minutes_day_end
    FROM Oxford_data_difference_day
),
Oxford_data_casetime AS (
    SELECT *
        , actual_end_case_datetime - actual_start_case_datetime AS case_duration
        , hour(case_duration) * 60 + minute(case_duration) AS case_duration_minutes
    FROM Oxford_data_turnovertime
),
Oxford_data_turnovertime_aggregate AS (
    SELECT session_key
        , first(specialism_name ORDER BY actual_day_sequence_number ASC) AS day_start_specialism
        , first(specialism_name ORDER BY day_end_overtime_indicator DESC, difference_day_end ASC) AS day_end_specialism
        , SUM(actual_turnovertime_minutes) AS total_turnovertime_minutes
        , SUM(case_duration_minutes) AS total_case_duration_minutes
        , COUNT(*) AS number_of_cases
    FROM Oxford_data_casetime ODF
    GROUP BY session_key
),
Oxford_data_joined AS (
    SELECT * EXCLUDE ODDM.session_key
    FROM Oxford_data_difference_minutes ODDM 
    INNER JOIN Oxford_data_turnovertime_aggregate ODTA ON ODDM.session_key = ODTA.session_key 
),
Oxford_data_format AS (
    SELECT 
        session_key,
        theatre,
        date,
        total_case_duration_minutes,
        total_turnovertime_minutes,
        number_of_cases,
        actual_start_case_datetime,
        expected_start_case_datetime,
        actual_end_case_datetime,
        expected_end_case_datetime,
        day_start_timeline_type,
        day_end_timeline_type,
        day_start_specialism,
        day_end_specialism,
        day_start_overtime_indicator,
        day_end_overtime_indicator,
        difference_day_start,
        difference_day_end,
        difference_minutes_day_start,
        difference_minutes_day_end
    FROM
        Oxford_data_joined
    WHERE 1=1 
        AND date IS NOT NULL 
        AND expected_end_case_datetime IS NOT NULL 
        AND actual_end_case_datetime IS NOT NULL
)
SELECT 
    *
FROM Oxford_data_format 

-- ORDER BY number_of_records_day_end DESC

In [ ]:
%%sql
WITH Oxford_normalized_difference AS (
    SELECT * REPLACE (
        IF(day_end_overtime_indicator !=  1, -1 * difference_minutes_day_end, difference_minutes_day_end) AS difference_minutes_day_end,
        IF(day_start_overtime_indicator !=  1, -1 * difference_minutes_day_start, difference_minutes_day_start) AS difference_minutes_day_start
    )
    FROM Oxford_parsed_efficiency_data
    WHERE     difference_minutes_day_end <= 120 AND difference_minutes_day_end >= -120
    AND
    difference_minutes_day_start <= 120 AND difference_minutes_day_start >= -120
)
SELECT *
FROM Oxford_normalized_difference

In [144]:
%%sql
SELECT *
FROM Oxford_data_turnovertime
WHERE surgeon_name IS NULL AND surgeon_surname IS NULL

Running query in 'duckdb'

organization_id,organization_name,theatre,case_key,session_key,actual_day_sequence_number,expected_day_sequence_number,surgeon_surname,surgeon_name,specialism_name,procedure_id,procedure_name,protocol_id,protocol_code,protocol_name,date,actual_start_case_datetime,expected_start_case_datetime,actual_end_case_datetime,expected_end_case_datatime,expected_previous_end_case_datetime,actual_previous_end_case_datetime,actual_turnovertime,expected_turnovertime,actual_turnovertime_minutes,expected_turnovertime_minutes
None,None,J-TH WW 07,21380615,554300,2,1,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-06-14,2024-06-14 09:54:00,2024-06-14 08:30:00,2024-06-14 10:32:00,2024-06-14 09:00:00,None,2024-06-14 09:44:00,0:10:00,None,10,None
None,None,J-TH WW 07,23005448,569753,3,4,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-09-28,2024-09-28 10:20:00,2024-09-28 10:35:00,2024-09-28 11:00:00,2024-09-28 11:20:00,2024-09-28 10:30:00,2024-09-28 10:10:00,0:10:00,0:05:00,10,5
None,None,C-TH Main 02,16471770,534086,4,3,None,None,Urology,None,None,None,None,None,2023-10-21,2023-10-21 14:10:00,2023-10-21 13:30:00,2023-10-21 15:33:00,2023-10-21 15:00:00,2023-10-21 13:15:00,2023-10-21 13:48:00,0:22:00,0:15:00,22,15
None,None,J-TH WW 07,19642083,557176,4,4,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-07-23,2024-07-23 12:10:00,2024-07-23 09:05:00,2024-07-23 12:49:00,2024-07-23 09:35:00,2024-07-23 09:05:00,2024-07-23 12:12:00,"-1 day, 23:58:00",0:00:00,-2,0
None,None,J-TH WW 07,22705164,560462,5,5,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-08-20,2024-08-20 13:40:00,2024-08-20 13:30:00,2024-08-20 14:21:00,2024-08-20 14:00:00,2024-08-20 11:30:00,2024-08-20 12:36:00,1:04:00,2:00:00,64,120
None,None,J-TH Gynae 01,18825503,551020,1,1,None,None,Gynaecology & Obstetrics,None,None,None,None,None,2024-04-13,2024-04-13 09:51:00,2024-04-13 08:30:00,2024-04-13 10:11:00,2024-04-13 09:10:00,None,None,None,None,None,None
None,None,J-TH WW 07,18885720,548449,2,3,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-04-21,2024-04-21 09:26:00,2024-04-21 08:00:00,2024-04-21 09:45:00,2024-04-21 08:30:00,2024-04-21 08:00:00,2024-04-21 09:22:00,0:04:00,0:00:00,4,0
None,None,J-TH WW 07,18553045,548032,6,2,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-03-17,2024-03-17 11:57:00,2024-03-17 08:00:00,2024-03-17 12:24:00,2024-03-17 08:30:00,2024-03-17 08:00:00,2024-03-17 11:44:00,0:13:00,0:00:00,13,0
None,None,C-TH Main 01,21622139,571898,2,2,None,None,Urology,None,None,None,None,None,2024-09-28,2024-09-28 11:30:00,2024-09-28 10:15:00,2024-09-28 12:05:00,2024-09-28 11:45:00,2024-09-28 10:00:00,2024-09-28 10:40:00,0:50:00,0:15:00,50,15
None,None,J-TH WW 07,13238040,539614,2,2,None,None,Ophthalmology & Ophthalmology (Surg),None,None,None,None,None,2024-01-11,2024-01-11 11:31:00,2024-01-11 08:30:00,2024-01-11 12:44:00,2024-01-11 08:55:00,2024-01-11 08:15:00,2024-01-11 10:59:00,0:32:00,0:15:00,32,15


In [ ]:
%%sql 
COPY Oxford_parsed_efficiency_data TO '../Oxford_parsed_efficiency_data_v1.csv'


In [89]:
df = conn.sql(
    """
    WITH Oxford_normalized_difference AS (
        SELECT * REPLACE (
            IF(day_end_overtime_indicator !=  1, -1 * difference_minutes_day_end, difference_minutes_day_end) AS difference_minutes_day_end,
            IF(day_start_overtime_indicator !=  1, -1 * difference_minutes_day_start, difference_minutes_day_start) AS difference_minutes_day_start
        )
        FROM Oxford_parsed_efficiency_data
        WHERE     difference_minutes_day_end <= 120 AND difference_minutes_day_end >= -120
        AND
        difference_minutes_day_start <= 120 AND difference_minutes_day_start >= -120
    )
    SELECT *
    FROM Oxford_normalized_difference
    """).df()

In [141]:
import altair as alt
import pandas as pd

df = df[~pd.isnull(df['expected_end_case_datetime'])]
alt \
    .Chart(df[['difference_minutes_day_end', 'difference_minutes_day_start']]) \
    .mark_bar() \
    .encode(
        x=alt.X('difference_minutes_day_end'),
        y=alt.Y('count()'),
        tooltip='difference_minutes_day_end'
    ).interactive()

alt.Chart(...)

### Visualization of the data

In [ ]:
import altair as alt
import pandas as pd

grouped_df = conn.sql(
    """
    SELECT 
        year(date) AS year, 
        month(date) AS month, 
        specialism_name, AVG(actual_turnovertime_minutes - expected_turnovertime_minutes) AS avg_diff_actual_turnovertime
    FROM Oxford_data_turnovertime
    WHERE expected_turnovertime IS NOT NULL AND actual_day_sequence_number = expected_day_sequence_number
    GROUP BY year(date), month(date), specialism_name
    HAVING COUNT()
    """
).df()
print(grouped_df.columns)
grouped_df['yearmonth'] =  pd.to_datetime(dict(year=grouped_df['year'], month=grouped_df['month'], day=1))
bar = alt.Chart().mark_bar().encode(x='yearmonth(yearmonth):T', y='average(avg_diff_actual_turnovertime):Q', color='specialism_name:N')
line = alt.Chart().mark_line().encode(x='yearmonth(date):T', y='average(avg_diff_actual_turnovertime):Q')
alt.layer(bar, data=grouped_df).facet(column='specialism_name:N')

In [ ]:
%%sql 
COPY Oxford_parsed_data TO 'Oxford_external_parsed_data_v1.csv'
